<h3>Import libraries:</h3>

In [1]:
import roboticstoolbox as rtb
from spatialmath import SE3, base, SO3
import matplotlib.pyplot as plt
from math import pi, degrees, radians
import numpy as np
from spatialmath.base import tr2rpy
from mpl_toolkits.mplot3d import Axes3D
import spatialgeometry as sg
import time
import math
from IPython.display import HTML

# https://stackoverflow.com/questions/44116194/import-a-function-from-another-ipynb-file
from ipynb.fs.full.inverse_kinematics_dnb_robot import *

%matplotlib qt

<h3>Define the robot:</h3>

In [48]:
# Define the lenghts
l1 = 10.3
l2 = 16.525
l3 = 15.645
l4 = 8.8
l5 = 7.4

# Adjust the lenghts to plot the robot properly with the library (otherwise, the axes are too small)
# https://la.mathworks.com/matlabcentral/answers/485353-frames-not-appearing-using-robotics-system-toolbox
# scaleFactor = 50
scaleFactor = 1

l1 /= scaleFactor
l2 /= scaleFactor
l3 /= scaleFactor
l4 /= scaleFactor
l5 /= scaleFactor

# Notes:
# 1. For a revolute joint the theta parameter of the link is ignored, and q used instead.
# 2. For a prismatic joint the d parameter of the link is ignored, and q used instead.
# 3. The joint offset parameter is added to q before computation of the transformation matrix.

# Robot's links
link1 = rtb.RevoluteDH(d=l1, a=0.0, alpha=pi/2, offset=-pi/2, qlim=[0,pi])
link2 = rtb.RevoluteDH(d=0.0, a=l2, alpha=0.0, offset=0.0, qlim=[0,pi])
link3 = rtb.RevoluteDH(d=0.0, a=l3, alpha=0.0, offset=-radians(152), qlim=[0,pi])
link4 = rtb.RevoluteDH(d=0.0, a=l4, alpha=0.0, offset=0.0, qlim=[0,pi])

# https://petercorke.github.io/robotics-toolbox-python/arm_dh.html#roboticstoolbox.robot.DHLink.PrismaticDH

# Create the robot
robot = rtb.DHRobot(
    links=[link1, link2, link3, link4],
    name="D&B Robot",
    tool = SE3.Ty(-l5)
    )

# Home angles
qHome = [radians(90), radians(90), radians(0), radians(62)]

# Set home position
robot.addconfiguration_attr('home',qHome)

print(robot)
print(f'Robot is in 1:{scaleFactor} scale.')

DHRobot: D&B Robot, 4 joints (RRRR), dynamics, standard DH parameters
┌────────────┬──────┬───────┬───────┬──────┬────────┐
│     θⱼ     │  dⱼ  │  aⱼ   │  ⍺ⱼ   │  q⁻  │   q⁺   │
├────────────┼──────┼───────┼───────┼──────┼────────┤
│  q1 - 90°  │ 10.3 │     0 │ 90.0° │ 0.0° │ 180.0° │
│  q2        │    0 │ 16.52 │  0.0° │ 0.0° │ 180.0° │
│  q3 - 152° │    0 │ 15.64 │  0.0° │ 0.0° │ 180.0° │
│  q4        │    0 │   8.8 │  0.0° │ 0.0° │ 180.0° │
└────────────┴──────┴───────┴───────┴──────┴────────┘

┌──────┬──────────────────────────────────────┐
│ tool │ t = 0, -7.4, 0; rpy/xyz = 0°, 0°, 0° │
└──────┴──────────────────────────────────────┘

┌──────┬──────┬──────┬─────┬──────┐
│ name │ q0   │ q1   │ q2  │ q3   │
├──────┼──────┼──────┼─────┼──────┤
│ home │  90° │  90° │  0° │  62° │
└──────┴──────┴──────┴─────┴──────┘

Robot is in 1:1 scale.


In [3]:
# Plot home position
fig1 = plt.figure(figsize=(8, 8))  # Adjust this for larger or smaller axes
robot.plot(q=robot.home, backend='pyplot', fig = fig1)
plt.show()

<h3>Kinematics tests:</h3>

In [4]:
# Compute the forward kinematics
fk1 = robot.fkine(qHome)

# Get the real coordinates
fk1Escaled = SE3.Rt(fk1.R, fk1.t * scaleFactor)
# This SE3.Rt method takes a rotation matrix R and a position vector t and creates the desired SE(3) transformation

print(fk1Escaled)

   1         0         0         16.14     
   0         0        -1         0         
   0         1         0         5.611     
   0         0         0         1         



In [5]:
# Compute the inverse kinematics
ik1 = robot.ikine_LM(fk1, joint_limits = True)

for i in range(len(ik1.q)):
    print(f'q{i+1} = {round(degrees(ik1.q[i]),2)}°.')

q1 = 90.0°.
q2 = 90.0°.
q3 = 0.0°.
q4 = 62.0°.


In [6]:
# We'll use our own inverse kinematics function which gives us easier control of the end effector's orientation

pxHome = fk1.A[:,3][0]
pyHome = fk1.A[:,3][1]
pzHome = fk1.A[:,3][2]

# The orientation of the end effector is defined with respect to the base frame (only the pitch angle can be controlled)
pitchAngle = radians(0)

qList = inverse_kinematics(pxHome, pyHome, pzHome, pitchAngle, scaleFactor)

for index,q in enumerate(qList):
    print(f'q{index+1}: {q}°.')

q1: 90.0°.
q2: 90.0°.
q3: 0.0°.
q4: 62.0°.


In [7]:
# Traslation vector and pitch angle for frame1
pxFrame1 = 19/scaleFactor
pyFrame1 = -8.2/scaleFactor
pzFrame1 = 0.2/scaleFactor
pitchAngleFrame1 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qFrame1 = inverse_kinematics(pxFrame1, pyFrame1, pzFrame1, pitchAngleFrame1, scaleFactor)
TFrame1 = robot.fkine([radians(i) for i in qFrame1])

print('Joint angles required to achieve the pose (orientation and position) of frame1:')
print(qFrame1)

print('\nHomogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:')
print(TFrame1)

# Extract RPY angles (mobile/current frame, ZYX convention)
rpyFrame1 = tr2rpy(TFrame1.R, order='zyx', unit='deg')

print('Euler angles in the ZYX convention with respect to the mobile/current frame:')
print(f'- Roll: {round(rpyFrame1[0],2)}°.')
print(f'- Pitch: {round(rpyFrame1[1],2)}°.')
print(f'- Yaw: {round(rpyFrame1[2],2)}°.')

# mobile/current frame, ZYX convention = fixed frame, XYZ convention
# https://mecademic.com/insights/academic-tutorials/space-orientation-euler-angles/
# https://bdaiinstitute.github.io/spatialmath-python/func_3d.html#spatialmath.base.transforms3d.tr2rpy

Joint angles required to achieve the pose (orientation and position) of frame1:
[66.66, 51.15, 16.45, 84.39]

Homogeneous transformation matrix (HTM) of {frame1} with respect to the base frame {0}:
   0.9182    0.0001603 -0.3962    0.38      
  -0.3962   -6.915e-05 -0.9182   -0.164     
  -0.0001745  1         0         0.003953  
   0         0         0         1         

Euler angles in the ZYX convention with respect to the mobile/current frame:
- Roll: 90.0°.
- Pitch: 0.01°.
- Yaw: -23.34°.


In [8]:
# Calculate the geometric jacobian with respect to the base frame
jacob1 = robot.jacob0(qFrame1)
print(jacob1)

[[ 6.19890728e-02  2.10426531e-01  4.83584738e-02 -1.21303790e-01]
 [ 5.08037414e-02 -2.56755609e-01 -5.90054369e-02  1.48010939e-01]
 [-7.65223838e-19 -8.01477716e-02 -2.89573330e-01 -1.27507460e-01]
 [-6.93889390e-17  7.73434764e-01  7.73434764e-01  7.73434764e-01]
 [ 0.00000000e+00  6.33875907e-01  6.33875907e-01  6.33875907e-01]
 [ 1.00000000e+00  6.12323400e-17  6.12323400e-17  6.12323400e-17]]


In [9]:
# Calculate the geometric jacobian with respect to the end-effector frame
jacob2 = robot.jacobe(qFrame1)
print(jacob2)

[[ 2.60208521e-18  4.27297446e-02  2.79292000e-01  1.48000000e-01]
 [-2.77555756e-17 -3.38822375e-01 -1.08022571e-01  1.76000000e-01]
 [ 8.01477716e-02 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
 [-9.93796042e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.11217922e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 6.12323400e-17  1.00000000e+00  1.00000000e+00  1.00000000e+00]]


In [10]:
# Calculate the analytical jacobian with respect to the base frame
jacob3 = robot.jacob0_analytical(qFrame1, representation='rpy/zyx')
print(jacob3)

[[ 6.19890728e-02  2.10426531e-01  4.83584738e-02 -1.21303790e-01]
 [ 5.08037414e-02 -2.56755609e-01 -5.90054369e-02  1.48010939e-01]
 [-7.65223838e-19 -8.01477716e-02 -2.89573330e-01 -1.27507460e-01]
 [ 3.95475620e-16  6.21724894e-15  6.21724894e-15  6.21724894e-15]
 [ 5.36678177e-17 -1.00000000e+00 -1.00000000e+00 -1.00000000e+00]
 [ 1.00000000e+00  6.21724894e-15  6.21724894e-15  6.21724894e-15]]


<h3>Trajectory test:</h3>

In [11]:
# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.trapezoidal

# https://la.mathworks.com/help/robotics/ref/trapveltraj.html
# https://la.mathworks.com/help/robotics/ug/choose-trajectories-for-manipulator-paths.html
# https://la.mathworks.com/help/robotics/ug/design-a-trajectory-with-velocity-limits-using-a-trapezoidal-velocity-profile.html
# https://medium.com/mathworks/trajectory-planning-for-robot-manipulators-522404efb6f0

# https://youtu.be/f3JVWqsDhbE?si=McvOoeyu82TyQrwv
# https://youtu.be/Fd7wjZDoh7g?si=ZXw3PyyvvAwSEzFu

# Servomotor MG996R ----> Operating speed: 0.17 s/60º (4.8 V), 0.14 s/60º (6 V) 
# 0.14 s/60º (6 V) --------> 7,14 °/s (max speed)

# Set the time required to complete the trajectory
timeStepsTraj1 = 100
totalTime = 10 # Seconds
timeTraj1 = np.linspace(0, totalTime, timeStepsTraj1)

In [12]:
# Compute the trajectory using a trapezoidal velocity profile
# tg = rtb.tools.trapezoidal(pxHome*scaleFactor, pxFrame1*scaleFactor, timeTraj1)
# tg.plot()

# Generate the trajectory using a trapezoidal interpolator
q0Traj1 = [i*scaleFactor for i in [pxHome, pyHome, pzHome]]
q1Traj1 = [i*scaleFactor for i in [pxFrame1, pyFrame1, pzFrame1]]
Traj1 = rtb.tools.trajectory.mtraj(rtb.tools.trapezoidal, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1.plot()

In [13]:
# Create a new figure for 3D plotting
fig2 = plt.figure()
ax = fig2.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1.s[:,0], Traj1.s[:,1], Traj1.s[:,2])

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a trapezoidal interpolator

In [14]:
# Let's repeat the same process using a quintic polynomial interpolator

# Generate the trajectory using a quintic polynomial interpolator
Traj1v2 = rtb.tools.trajectory.mtraj(rtb.tools.quintic, q0Traj1, q1Traj1, timeTraj1)

# Plot the position, velocity and acceleration data of the trajectory
Traj1v2.plot()

In [15]:
# Create a new figure for 3D plotting
fig3 = plt.figure()
ax = fig3.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v2.s[:,0], Traj1v2.s[:,1], Traj1v2.s[:,2])

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We got a line in the cartesian space using a quintic polynomial interpolator

In [16]:
# Let's do it again with initial and final velocity different from zero (impractical in this case, it's just for testing)

# Set the velocities (they could be different for each axis)
dq0Traj1v3 = 0.5
dq1Traj1v3 = 1

Traj1v3x = rtb.tools.trajectory.quintic(q0Traj1[0], q1Traj1[0], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3y = rtb.tools.trajectory.quintic(q0Traj1[1], q1Traj1[1], timeTraj1, dq0Traj1v3, dq1Traj1v3)
Traj1v3z = rtb.tools.trajectory.quintic(q0Traj1[2], q1Traj1[2], timeTraj1, dq0Traj1v3, dq1Traj1v3)

# Create a new figure for 3D plotting
fig4 = plt.figure()
ax = fig4.add_subplot(111, projection='3d')

# Plot the 3D trajectory in the workspace
ax.plot(Traj1v3x.s, Traj1v3y.s, Traj1v3z.s)

# Add labels and title
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_title('Task-space trajectory')

# Show the plot
plt.show()

# We don't get a line in the cartesian space using a polynomial interpolator if the initial and final velocities are different from zero
# This also happens between viapoints

In [17]:
# We've only interpolated the position of the robot, but it's also required an orientation interpolation
# Position and orientation interpolation are two different issues, therefore they're done separately

# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.ctraj
# We'll handle position and orientation interpolation through the ctraj function

# Set the time required to complete the trajectory
timeStepsTraj2 = 120
totalTime2 = 20 # Seconds
timeTraj2 = np.linspace(0, totalTime2, timeStepsTraj2)

# Compute the Cartesian trajectory between two poses (position + orientation)
traj2 = rtb.tools.trajectory.ctraj(fk1, TFrame1, timeTraj2)

# Afterwards, compute the inverse kinematics for each pose along the trajectory
# These joint configurations can be fed into a control system to execute and control the movements of the robotic arm

# qTraj2 = robot.ikine_LM(traj2)

# Plot the trajectory with the library
# robot.plot(qTraj2.q, backend='pyplot')
# plt.show()

In [18]:
# Plot the evolution of each position vector component in the trajectory

xTraj2 = traj2.x
yTraj2 = traj2.y
zTraj2 = traj2.z

fig6, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True)
fig6.suptitle('Evolution of each position vector component in the trajectory')

ax1.plot(timeTraj2, xTraj2)
ax1.set_ylabel('x')

ax2.plot(timeTraj2, yTraj2)
ax2.set_ylabel('y')

ax3.plot(timeTraj2, zTraj2)
ax3.set_ylabel('z')
ax3.set_xlabel('t')

plt.tight_layout()
plt.show()

In [19]:
# Plot the evolution of each position vector component's velocity in the trajectory

# Get the time step
deltaT = timeTraj2[5] - timeTraj2[4]

deltaX = np.hstack((0,np.diff(xTraj2, axis = 0)))
deltaY = np.hstack((0,np.diff(yTraj2, axis = 0)))
deltaZ = np.hstack((0,np.diff(zTraj2, axis = 0)))
# Add zero at the start of the array to match the position array size

velX = deltaX/deltaT
velY = deltaY/deltaT
velZ = deltaZ/deltaT

fig7, (ax1, ax2, ax3) = plt.subplots(3,1, sharex=True)
fig7.suptitle('Evolution of each position vector components velocity in the trajectory')

ax1.plot(timeTraj2, velX)
ax1.set_ylabel('x')

ax2.plot(timeTraj2, velY)
ax2.set_ylabel('y')

ax3.plot(timeTraj2, velZ)
ax3.set_ylabel('z')
ax3.set_xlabel('t')

plt.tight_layout()
plt.show()

<h3>Resolved-Rate Motion Control:</h3>

In [20]:
# https://youtu.be/sc5DZOkNvTc?si=JpxpIKR4r8iLLvUk
# https://youtu.be/1GaxV8x4YHA?si=SVk1Yh55wbxU7_Xb
# https://youtu.be/AgQ_Bybl_9k?si=KLwMJ_HOi7Fy8zhD
# https://youtu.be/5lZD93jh2m0?si=QN1pOT8hGjsfnb5M
# https://youtu.be/FU9jC1lHt_c?si=89MTK9BMjQouzqFE

<h4>Method 1 (RRMC without velocity profile nor cartesian interpolation):</h4>

In [21]:
# Based on the following notebook:
# https://github.com/jhavl/dkt/blob/main/Part%201/3%20Resolved-Rate%20Motion%20Control.ipynb
# https://arxiv.org/pdf/2207.01796

In [22]:
# https://youtu.be/-TUTqVOGSa8?si=3OldHsu_qE9i22FQ
# https://youtu.be/s0NC85TAPaA?si=zr1ObUN9i66GhVMp
# https://youtu.be/axEadpYSOv8?si=grupjc4v4PHLdA9d

def error(currentPose, desiredPose):

    # Returns the error between the desired and the current pose (position: vector, rotation: angle-axis notation)
    # currentPose and desiredPose --> np.ndarray with shape = (4,4)
    # currentPose and desiredPose are the poses of the end effector relative to the base frame
    
    # Error array
    e = np.empty(6)

    # The position error
    e[:3] = desiredPose[:3, -1] - currentPose[:3, -1]

    # The rotation error expressed as rotation matrix
    R = desiredPose[:3, :3] @ currentPose[:3, :3].T

    # Transform the rotation matrix to its Euler vector equivalent
    
    li = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

    if np.linalg.norm(li) < 1e-6:
        # If li is a zero vector (or very close to it)

        # Diagonal matrix case
        if np.trace(R) > 0:
            # (1,1,1) case
            a = np.zeros((3,))
        else:
            a = np.pi / 2 * (np.diag(R) + 1)
    else:
        # Non-diagonal matrix case
        ln = np.linalg.norm(li)
        a = math.atan2(ln, np.trace(R) - 1) * li / ln
    
    # Append the rotation error expressed in the angle-axis representation in the error array
    e[3:] = a

    # Return the error
    return e

In [23]:
def p_servo(currentPose, desiredPose, gain, threshold = 0.001):
    
    # Position-based servoing

    # Returns the end-effector velocity in the cartesian space which will cause the robot to approach the desired pose

    # currentPose: The current pose of the end-effecor in the base frame (ndarray)
    # desiredPose: The desired pose of the end-effecor in the base frame (ndarray)
    # gain: The gain for the proportional controller. A vector corresponding to each Cartesian axis (array-like)
    # threshold: The threshold or tolerance of the final error between the robot's pose and desired pose (float)

    # v: The velocity of the end-effector which will casue the robot to approach desiredPose (ndarray(6))
    # arrived: True if the robot is within the threshold of the final pose (bool)
    
    if isinstance(currentPose, SE3):
        currentPose = currentPose.A

    if isinstance(desiredPose, SE3):
        desiredPose = desiredPose.A

    # Calculate the pose error vector
    e = error(currentPose, desiredPose)

    # Construct our gain diagonal matrix
    if base.isscalar(gain):
        k = gain * np.eye(6)
    else:
        k = np.diag(gain)

    # Calculate our desired end-effector velocity
    v = k @ e

    # Check if we have arrived
    arrived = True if np.sum(np.abs(e)) < threshold else False

    # Tuple[np.ndarray, bool]
    return v, arrived

In [24]:
# https://github.com/petercorke/robotics-toolbox-python/blob/master/roboticstoolbox/examples/RRMC.py

env = rtb.backends.PyPlot.PyPlot()
env.launch('Resolved-Rate Motion Control Example')

arrived = False

env.add(robot)

# Set the current pose
robot.q = qHome

# Desired pose
pxFrame2 = 37/scaleFactor
pyFrame2 = -8.2/scaleFactor
pzFrame2 = 0.2/scaleFactor
pitchAngleFrame2 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qFrame2 = inverse_kinematics(pxFrame2, pyFrame2, pzFrame2, pitchAngleFrame2, scaleFactor)
TFrame2 = robot.fkine([radians(i) for i in qFrame2])

# Proportional control gain
gain = 3

# Time step
dt = 0.05

# Set the maximum end-effector velocity
vMax = 1.0

# Lists to save the values obtained in each time step
posesObtained = []
cartesianVelocitiesObtained = []
jointPositionsObtained = []
jointVelocitiesObtained = []

while not arrived:

    start = time.time()
    
    # HTM involved
    currentPose = robot.fkine(robot.q)
    desiredPose = TFrame2
    
    # Cartesian velocity required (both linear and angular velocities)
    v, arrived = p_servo(currentPose, desiredPose, gain)
    
    # Calculate the magnitude of the end-effector velocity
    vNorm = np.linalg.norm(v)

    # If the end-effector velocity exceeds the maximum allowed
    if vNorm > vMax:
        v = (vMax / vNorm) * v

    # Map it to joint velocities using the geometric jacobian relative to the base frame
    qd = np.linalg.pinv(robot.jacob0(robot.q)) @ v
    
    # Apply the calculated joint velocity to the robot's actuators for movement
    robot.qd = qd
    
    # Append the values obtained in this iteration
    posesObtained.append(currentPose)
    cartesianVelocitiesObtained.append(v)
    jointPositionsObtained.append(robot.q)
    jointVelocitiesObtained.append(qd)
    
    env.step(dt)
    stop = time.time()
    
    if stop - start < dt:
        time.sleep(dt - (stop - start))

# robot.plot(q=[radians(i) for i in qFrame2], backend='pyplot')

In [25]:
# Create a new figure for 3D plotting
fig5 = plt.figure()
ax1 = fig5.add_subplot(111, projection='3d')
# ax2 = fig5.add_subplot(122, projection='3d')

# Plot the 3D trajectory for the translational motion in the workspace
for htm in posesObtained:
    ax1.scatter(htm.A[0,3], htm.A[1,3], htm.A[2,3])

# Plot the desired pose
ax1.scatter(TFrame2.A[0,3], TFrame2.A[1,3], TFrame2.A[2,3], marker="*", linewidths=3, c='r')

# Add labels and title
ax1.set_xlabel('X Label')
ax1.set_ylabel('Y Label')
ax1.set_zlabel('Z Label')
ax1.set_title('Task-space trajectory')

# for vel in cartesianVelocitiesObtained:
#     ax2.scatter(vel[0], vel[1], vel[2])

# Show the plot
plt.show()

In [26]:
# Time spent in the trajectory
t = np.arange(0, len(cartesianVelocitiesObtained)*dt, dt)

fig8, (ax1, ax2) = plt.subplots(2,1, sharex=True)
fig8.suptitle('Trajectory')

ax1.plot(t, [pose.x for pose in posesObtained], label='x')
ax1.plot(t, [pose.y for pose in posesObtained], label='y')
ax1.plot(t, [pose.z for pose in posesObtained], label='z')
ax1.set_ylabel('Position')
ax1.legend()

ax2.plot(t, [velocity[0] for velocity in cartesianVelocitiesObtained], label='vx')
ax2.plot(t, [velocity[1] for velocity in cartesianVelocitiesObtained], label='vy')
ax2.plot(t, [velocity[2] for velocity in cartesianVelocitiesObtained], label='vz')
ax2.set_ylabel('Velocity')
ax2.set_xlabel('Time')
ax2.legend()

plt.tight_layout()
plt.show()

<h4>Method 2 (RRMC with position and velocity profiles):</h4>

In [49]:
# https://www.youtube.com/watch?v=5lZD93jh2m0

In [90]:
def rotation_matrix_to_angle_axis_repr(R):
    
    # Transform the rotation matrix to its Euler vector equivalent
    
    li = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

    if np.linalg.norm(li) < 1e-6:
        # If li is a zero vector (or very close to it)

        # Diagonal matrix case
        if np.trace(R) > 0:
            # (1,1,1) case
            a = np.zeros((3,))
        else:
            a = np.pi / 2 * (np.diag(R) + 1)
    else:
        # Non-diagonal matrix case
        ln = np.linalg.norm(li)
        a = math.atan2(ln, np.trace(R) - 1) * li / ln

    return a

def pose_error(currentPose, desiredPose):

    # Returns the error between the desired and the current pose (position: vector, orientation: angle-axis notation)
    # currentPose and desiredPose --> np.ndarray with shape = (4,4)
    # currentPose and desiredPose are the poses of the end effector relative to the base frame
    
    if isinstance(currentPose, SE3):
        currentPose = currentPose.A

    if isinstance(desiredPose, SE3):
        desiredPose = desiredPose.A

    # Error array
    e = np.empty(6)

    # The position error
    e[:3] = desiredPose[:3, -1] - currentPose[:3, -1]

    # The rotation error expressed as rotation matrix
    R = desiredPose[:3, :3] @ currentPose[:3, :3].T

    # Transform the rotation matrix to its Euler vector equivalent
    a = rotation_matrix_to_angle_axis_repr(R)
    
    # Append the rotation error expressed in the angle-axis representation in the error array
    e[3:] = a

    # Return the error
    # e.shape = (6,)
    return e

In [178]:
# Get the HTM for the initial configuration of the robot in the trajectory
T0 = robot.fkine(qHome)

# Get the desired pose
pxT1 = 37/scaleFactor
pyT1 = -8.2/scaleFactor
pzT1 = 0.2/scaleFactor
pitchAngleT1 = radians(0)

# Its orientation needs to be determined; it's only known that the pitch angle, in the RPY convention, is equal to zero (parallel to the surface)
qT1 = inverse_kinematics(pxT1, pyT1, pzT1, pitchAngleT1, scaleFactor)
T1 = robot.fkine([radians(i) for i in qT1])

# type(T0) and type(T1) ----> spatialmath.pose3d.SE3

# Set time step in seconds
dt = 0.1

# Total time in seconds
totalTime = 8

# Time array
trajTime = np.arange(0, totalTime, dt)

# Number of steps
stepsNum = len(trajTime)

# https://petercorke.github.io/robotics-toolbox-python/arm_trajectory.html#roboticstoolbox.tools.trajectory.trapezoidal
# Generate the trapezoidal trajectory for the translational components
x = rtb.tools.trajectory.trapezoidal(T0.A[0,3], T1.A[0,3], trajTime)
y = rtb.tools.trajectory.trapezoidal(T0.A[1,3], T1.A[1,3], trajTime)
z = rtb.tools.trajectory.trapezoidal(T0.A[2,3], T1.A[2,3], trajTime)

# https://bdaiinstitute.github.io/spatialmath-python/3d_orient_SO3.html#spatialmath.pose3d.SO3.interp
# https://bdaiinstitute.github.io/spatialmath-python/3d_pose_SE3.html#spatialmath.pose3d.SE3.interp
# Generate the trajectory for the orientation component
rotMatrix = SO3(T0.R).interp(SO3(T1.R), stepsNum)

# Convert the rotation matrices to angle axis representation 
angularPosition = [rotation_matrix_to_angle_axis_repr(rotMat) for rotMat in rotMatrix.A]

# Velocities of the translational components along the trajectory (linear velocities)
vx = x.qd
vy = y.qd
vz = z.qd

# Calculate the velocity of the rotational component along the trajectory
# https://theorangeduck.com/page/exponential-map-angle-axis-angular-velocity#:~:text=The%20angle%2Daxis%20representation%3A%20A,rotation%20with%20respect%20to%20time.

# Differenciate between rotation matrices
deltaRotMatrix = [rotMatrix.A[i+1]*np.transpose(rotMatrix.A[i]) for i in range(len(rotMatrix.A)-1)]

# Convert the rotation matrices to angle axis representation 
deltaAngleAxis = [rotation_matrix_to_angle_axis_repr(rotMat) for rotMat in deltaRotMatrix]
# deltaAngleAxis.append(np.array([0,0,0], 'float64'))
deltaAngleAxis.append(deltaAngleAxis[-1])

# Obtain the angular velocities
# https://www.euclideanspace.com/physics/kinematics/angularvelocity/

w = [angleAxisRepr/dt for angleAxisRepr in deltaAngleAxis]

In [179]:
# Plot all the calculated parameters

fig9, axes = plt.subplots(2,2) 
# plt.subplots(2, 2) returns a 2x2 array of axes

# Unpack the axes array into individual variables
(ax1, ax2), (ax3, ax4) = axes

fig9.suptitle('Trajectory')

ax1.plot(trajTime, x.q, label='x')
ax1.plot(trajTime, y.q, label='y')
ax1.plot(trajTime, z.q, label='z')
ax1.set_ylabel('Position')
ax1.legend()

ax2.plot(trajTime, x.qd, label='vx')
ax2.plot(trajTime, y.qd, label='vy')
ax2.plot(trajTime, z.qd, label='vz')
ax2.set_ylabel('Linear velocity')
ax2.set_xlabel('Time')
ax2.legend()

ax3.plot(trajTime, [angPos[0] for angPos in angularPosition], label='x-axis rotation')
ax3.plot(trajTime, [angPos[1] for angPos in angularPosition], label='y-axis rotation')
ax3.plot(trajTime, [angPos[2] for angPos in angularPosition], label='z-axis rotation')
ax3.set_ylabel('Angular position')
ax3.set_xlabel('Time')
ax3.legend()

ax4.plot(trajTime, [wxyz[0] for wxyz in w], label='wx')
ax4.plot(trajTime, [wxyz[1] for wxyz in w], label='wy')
ax4.plot(trajTime, [wxyz[2] for wxyz in w], label='wz')
ax4.set_ylabel('Angular velocity')
ax4.set_xlabel('Time')
ax4.legend()


plt.tight_layout()
plt.show()

In [180]:
# HTML(rotMatrix.animate(movie=True, dim=1.5))

In [181]:
# Create the homogeneous transformation matrices using the rotation matrices and translation vectors from the trajectory
XRef = [SE3.Rt(rotMatrix[j], np.array([x.q[j], y.q[j], z.q[j]])) for j in range(len(rotMatrix))]

# Organize the velocities to ease its manipulation
XdRef = [np.array([vx[i], vy[i], vz[i], w[i][0], w[i][1], w[i][2]]) for i in range(len(w))]

In [182]:
# https://github.com/petercorke/robotics-toolbox-python/blob/master/roboticstoolbox/examples/RRMC.py

env2 = rtb.backends.PyPlot.PyPlot()
env2.launch('Resolved-Rate Motion Control Example')

arrived = False

env2.add(robot)

# Set the starting pose in the robot
robot.q = qHome

# Proportional control gain
gain = 10

# Construct our gain diagonal matrix
if base.isscalar(gain):
    kp = gain * np.eye(6)
else:
    kp = np.diag(gain)
    
threshold = 10

i = 0 # Auxiliar variable

# Lists to save the values obtained in each time step
posesObtained = []
cartesianVelocitiesObtained = []
jointPositionsObtained = []
jointVelocitiesObtained = []

while not arrived:
    
    start = time.time()
    
    # Current robot's pose
    currentPose = robot.fkine(robot.q)

    if i < len(XRef):
        # Desired pose  in this specific time step:
        desiredPose = XRef[i]
        actualXdRef = XdRef[i]
    else:
        # If all trajectory points have been processed, ensure the robot's final pose is achieved correctly
        desiredPose = XRef[-1]
        actualXdRef = XdRef[-1]
        
    # Compute the position error
    poseError = pose_error(currentPose, desiredPose)
    
    # Compute the combined term (feedforward + feedback)
    referenceVelocityAdjusted = actualXdRef + (kp @ poseError)

    # Use this term to calculate joint velocities using the geometric jacobian relative to the base frame
    qd = np.linalg.pinv(robot.jacob0(robot.q)) @ referenceVelocityAdjusted
    print(qd)
    
    # Apply the calculated joint velocity to the robot's actuators for movement
    robot.qd = qd
    
    if i >= len(XRef):
        # Check if the robotic arm has reached the desired point
        arrived = True if np.sum(np.abs(poseError)) < threshold else False

    # Update the auxiliar variable
    i += 1
    
    # Append the values obtained in this iteration
    posesObtained.append(currentPose)
    cartesianVelocitiesObtained.append(referenceVelocityAdjusted)
    jointPositionsObtained.append(robot.q)
    jointVelocitiesObtained.append(qd)

    env2.step(dt)
    stop = time.time()
    
    # Complete the time step
    if stop - start < dt:
        time.sleep(dt - (stop - start))

[-1.49193338e-14  2.16170023e-12  2.51559743e-12 -8.58118427e-12]
[-0.00557955 -0.02031166  0.01234182  0.00796984]
[-0.00919517 -0.042288    0.01064439  0.0471236 ]
[-0.012761   -0.06199289  0.01171541  0.07717999]
[-0.01625699 -0.08162749  0.01310371  0.10669723]
[-0.01966493 -0.10117188  0.01493904  0.13544256]
[-0.0229677  -0.12059734  0.01732472  0.16323372]
[-0.02614949 -0.13984796  0.02035987  0.18986661]
[-0.02919597 -0.15883766  0.02413776  0.21511459]
[-0.03209443 -0.17745002  0.02874319  0.23873236]
[-0.03483387 -0.19554001  0.03424991  0.2604614 ]
[-0.03740509 -0.21293818  0.04071818  0.28003719]
[-0.03980068 -0.22945739  0.04819252  0.29719864]
[-0.04201507 -0.24490213  0.05669972  0.31169957]
[-0.04404449 -0.25907982  0.06624743  0.32332156]
[-0.04588689 -0.27181349  0.07682344  0.33188728]
[-0.04754187 -0.28295457  0.08839588  0.33727295]
[-0.0490106  -0.29239454  0.10091445  0.33941849]
[-0.05029565 -0.30007409  0.11431273  0.33833412]
[-0.0514009  -0.30598864  0.128511

In [184]:
# Create a new figure for 3D plotting
fig10 = plt.figure()
ax1 = fig10.add_subplot(111, projection='3d')

# Plot the 3D trajectory for the translational motion in the workspace
for htm in posesObtained:
    ax1.scatter(htm.A[0,3], htm.A[1,3], htm.A[2,3])

# Plot the desired pose
ax1.scatter(T1.A[0,3], T1.A[1,3], T1.A[2,3], marker="*", linewidths=3, c='r')

# Add labels and title
ax1.set_xlabel('X Label')
ax1.set_ylabel('Y Label')
ax1.set_zlabel('Z Label')
ax1.set_title('Task-space trajectory')

# Show the plot
plt.show()

# fig10, (ax1, ax2) = plt.subplots(2,1, sharex=True)
# fig10.suptitle('Trajectory')

# ax1.plot(trajTime, [pose.x for pose in posesObtained], label='x')
# ax1.plot(trajTime, [pose.y for pose in posesObtained], label='y')
# ax1.plot(trajTime, [pose.z for pose in posesObtained], label='z')
# ax1.set_ylabel('Position')
# ax1.legend()

# ax2.plot(trajTime, [velocity[0] for velocity in cartesianVelocitiesObtained], label='vx')
# ax2.plot(trajTime, [velocity[1] for velocity in cartesianVelocitiesObtained], label='vy')
# ax2.plot(trajTime, [velocity[2] for velocity in cartesianVelocitiesObtained], label='vz')
# ax2.set_ylabel('Velocity')
# ax2.set_xlabel('Time')
# ax2.legend()

# plt.tight_layout()
# plt.show()